In [1]:
import dask.dataframe as dd

import pandas as pd

In [37]:
ddf = dd.read_json("people1.json", orient="split")

In [38]:
ddf.compute()

,name,age,car
0,John,30,honda
1,Sally,54,kia


In [31]:
ddf = dd.read_json("people2.json", orient="split")

In [32]:
ddf.compute()

,name,age,car
0,John,30,honda
1,Sally,54,kia


In [33]:
ddf = dd.read_json("people3.json", lines=True)

In [34]:
ddf.compute()

,name,age,car
0,John,30,honda
1,Sally,54,kia


In [41]:
ddf = dd.read_json("students.json", orient="split")

In [42]:
ddf.compute()

,name,age,scores
0,Li,15,"[34, 99, 86]"
1,Qu,18,"[99, 100, 87]"


In [43]:
ddf.dtypes

name      object
age        int64
scores    object
dtype: object

In [47]:
ddf["average_score"] = ddf["scores"].apply(
    lambda x: sum(x) / len(x), meta=("average_score", "float64")
)

In [48]:
ddf.compute()

,name,age,scores,average_score
0,Li,15,"[34, 99, 86]",73.000000
1,Qu,18,"[99, 100, 87]",95.333333


In [121]:
ddf = dd.read_json("students2.json", orient="split")

In [122]:
ddf["exam_subject"] = ddf.exam.apply(lambda x: x["subject"], meta=("exam_subject", "object"))

In [123]:
ddf.compute()

,name,age,exam,exam_subject
0,george,16,"{'subject': 'geometry', 'score': 56}",geometry
1,nora,7,"{'subject': 'geometry', 'score': 87}",geometry


In [125]:
ddf["exam_score"] = ddf.exam.apply(lambda x: x["score"], meta=("exam_score", "int64"))

In [126]:
ddf.compute()

,name,age,exam,exam_subject,exam_score
0,george,16,"{'subject': 'geometry', 'score': 56}",geometry,56
1,nora,7,"{'subject': 'geometry', 'score': 87}",geometry,87


In [42]:
ddf = dd.read_json("students3.json", orient="split")

In [35]:
df = pd.read_json("students3.json", orient="split")

In [36]:
df

,name,age,exams
0,george,16,"[{'subject': 'geometry', 'score': 56}, {'subje..."
1,nora,7,"[{'subject': 'geometry', 'score': 87}, {'subje..."


In [39]:
exploded = df.explode("exams")

In [40]:
exploded

,name,age,exams
0,george,16,"{'subject': 'geometry', 'score': 56}"
0,george,16,"{'subject': 'poetry', 'score': 88}"
1,nora,7,"{'subject': 'geometry', 'score': 87}"
1,nora,7,"{'subject': 'poetry', 'score': 94}"


In [41]:
pd.concat([exploded[["name", "age"]].reset_index(drop=True), pd.json_normalize(exploded["exams"])], axis=1)

,name,age,subject,score
0,george,16,geometry,56
1,george,16,poetry,88
2,nora,7,geometry,87
3,nora,7,poetry,94


In [43]:
exploded = ddf.explode("exams")

In [44]:
dd.concat([exploded[["name", "age"]].reset_index(drop=True), dd.json_normalize(exploded["exams"])], axis=1)

AttributeError: module 'dask.dataframe' has no attribute 'json_normalize'

In [19]:
df.explode("exams")

,name,age,exams
0,george,16,"{'subject': 'geometry', 'score': 56}"
0,george,16,"{'subject': 'poetry', 'score': 88}"
1,nora,7,"{'subject': 'geometry', 'score': 87}"
1,nora,7,"{'subject': 'poetry', 'score': 94}"


In [10]:
df["exams"]

0    [{'subject': 'geometry', 'score': 56}, {'subje...
1    [{'subject': 'geometry', 'score': 87}, {'subje...
Name: exams, dtype: object

In [16]:
pd.json_normalize(df["exam"])

,subject,score
0,geometry,56
1,geometry,87


## Stackoverflow example

Here's the question on SO: https://stackoverflow.com/questions/49671693/pandas-dataframe-normalize-one-json-column-and-merge-with-other-columns/71847538#71847538

In [21]:
import json
df_actions = pd.DataFrame(columns=['id', 'actions'])
rows = [[12,json.loads('[{"type": "a","value": "17"},{"type": "b","value": "19"}]')],
   [15, json.loads('[{"type": "a","value": "1"},{"type": "b","value": "3"},{"type": "c","value": "5"}]')]]
df_actions.loc[0] = rows[0]
df_actions.loc[1] = rows[1]

In [22]:
df_actions

,id,actions
0,12,"[{'type': 'a', 'value': '17'}, {'type': 'b', '..."
1,15,"[{'type': 'a', 'value': '1'}, {'type': 'b', 'v..."


In [25]:
exploded = df_actions.explode("actions")

In [26]:
exploded

,id,actions
0,12,"{'type': 'a', 'value': '17'}"
0,12,"{'type': 'b', 'value': '19'}"
1,15,"{'type': 'a', 'value': '1'}"
1,15,"{'type': 'b', 'value': '3'}"
1,15,"{'type': 'c', 'value': '5'}"


In [34]:
pd.concat([exploded["id"].reset_index(drop=True), pd.json_normalize(exploded["actions"])], axis=1)

,id,type,value
0,12,a,17
1,12,b,19
2,15,a,1
3,15,b,3
4,15,c,5


In [28]:
exploded["id"]

0    12
0    12
1    15
1    15
1    15
Name: id, dtype: int64

In [29]:
pd.json_normalize(exploded["actions"])

,type,value
0,a,17
1,b,19
2,a,1
3,b,3
4,c,5


## Reading multiple JSON files

In [45]:
ddf = dd.read_json("./json-data/pets*.json", orient="split")

In [47]:
print(ddf.compute())

     name species   color
0   Triss     cat  orange
1    Dale     dog   brown
0   Gregg    bird   green
1  Weston    wolf    grey
